In [1]:
from resnet import *
from train_util import *
from FP_layers import *

import torch
import numpy as np
import matplotlib.pyplot as plt

import torchvision.transforms as transforms
import torchvision
import torch.nn as nn
import torch.optim as optim

import torch.multiprocessing as mp
from torch.utils.data.distributed import DistributedSampler
from torch.nn.parallel import DistributedDataParallel as DDP
from torch.distributed import init_process_group, destroy_process_group

import time
import os
# os.environ["PYTORCH_CUDA_ALLOC_CONF"] = "max_split_size_mb:512"

device = 'cuda' if torch.cuda.is_available() else 'cpu'
# print(device, 'torch.distributed.is_available():', torch.distributed.is_available())
print(device)
# torch.distributed.init_process_group()

cuda


In [2]:
# from lars.lars import LARS
# head = nn.Sequential(FP_Linear(64, 64, Nbits=None), nn.ReLU(True), FP_Linear(64, 64, Nbits=None))

# model = nn.parallel.DistributedDataParallel(ResNetCIFAR(head_g=head))
# model = nn.DataParallel(ResNetCIFAR(head_g=head))

batch_size = int(1024)
# world_size = torch.cuda.device_count()
# print(world_size)
# mp.spawn(train, args=(world_size, 100, batch_size, 0.3*batch_size/256, 1e-6, head, 50,), nprocs=world_size, join='True')


# rank, world_size, epochs, batch_size, lr, reg, head, log_every_n=50):
# train(model, epochs=100, batch_size=batch_size, lr=0.3*batch_size/256, reg=1e-6, world_size=world_size, log_every_n=50)
# net, epochs, batch_size, lr, reg, rank, world_size, log_every_n=50
# optimizer = LARS(model.parameters(), lr=0.1, momentum=0.9)
# optimizer.zero_grad()
# loss_fn(model(input), target).backward()
# optimizer.step()

In [3]:
# model = nn.parallel.DistributedDataParallel(ResNetCIFAR(head_g=head))


# head = nn.Sequential(FP_Linear(64, 64, Nbits=None), nn.ReLU(True), FP_Linear(64, 64, Nbits=None))
head = nn.Sequential(nn.Linear(64, 64), nn.ReLU(True), nn.Linear(64, 64))
# model = nn.DataParallel(ResNetCIFAR_mine(num_layers=50)).to(device)
model = ResNetCIFAR(head_g=head, num_layers=50).to(device)
# train_no_DDP(model, epochs=1000, batch_size=batch_size, lr=0.3*batch_size/256, reg=1e-6, log_every_n=20)
train_nt_xet_class(model, epochs=1000, batch_size=batch_size, lr=0.3*batch_size/256, reg=1e-6, log_every_n=20)

# # b = torch.rand((64, 3, 28, 28))
# head = nn.Sequential(FP_Linear(64, 64, Nbits=None), nn.ReLU(True), FP_Linear(64, 64, Nbits=None))

# y = model(b)

# # xcs = F.cosine_similarity(y[None, :, :], y[:,None,:], dim=-1)
# # ce_loss = F.cross_entropy(xcs / temperature, target, reduction="mean")
# # diff_sum = 0
# # for rr, cc in enumerate(range(128)):
#     # diff_sum += xcs[rr, cc] - F.cosine_similarity(y[rr, :], y[cc, :], dim=-1)

TypeError: __init__() missing 1 required positional argument: 'head_g'

In [ ]:
# head = nn.Sequential(FP_Linear(64, 64, Nbits=None), nn.ReLU(True), FP_Linear(64, 64, Nbits=None))
# model = ResNetCIFAR(head_g=head).to(device)
# trainset = torchvision.datasets.CIFAR10(root='./data', train=True, download=False, transform=transforms.ToTensor())
# trainloader = torch.utils.data.DataLoader(trainset, batch_size=128, shuffle=True, num_workers=1)
# b = None
# for idx, (inputs, label) in enumerate(trainloader):
#     print(inputs.shape)
#     b = augment_data(inputs)
#     break

In [ ]:
# idx = -2
# plt.imshow(b[idx, 0, :, :])
# plt.show()
# plt.imshow(b[idx+1, 0, :, :])
# plt.show()
crit = ContrastiveLoss(16)
# data_1 = torch.zeros((16, 3))
# for i in range(8):
#     data_1[2*i, :] = i
#     data_1[2*i+1, :] = i

data_1 = torch.zeros((16, 3))
data_2 = torch.zeros((16, 3))
for i in range(16):
    data_1[i, :] = np.sqrt(i)
    # data_2[i, :] = torch.exp(torch.arange(3))
    # data_2[i, :] = torch.arange(3)
    # data_2[i, :] = i
    data_2[i, :] = 0
crit(data_1, data_2)